# La Commune de Paris 1871 en liberté -  Les communardes et communards dans des cimetières

In [1]:
from extraction_wd import *
from IPython.display import HTML

endpoint_url:str = "https://query.wikidata.org/sparql"

## Généralités
Les communard·e·s étant touz* décédæ, c'est notamment dans les cimetières, quand les tombes existent, qu'on peut leur rendre hommage. N'oublions pas que beaucoup sont dans des fossés communes anonymes.

En termes de méthode wikidata nous propose la propriété *lieu de sépulture* (P119). Selon les précisions cela peut être une ville ou un cimetière. Cet élément possède une propriété *coordonnées géographiques* (P625).

Pour certaines personnes, la propriété photo de la sépulture est rempli avec une photo de communs.
Pour certaines personnes il existe même un élément pour leur leur tombe. 

On peut donc connaître le nombre de personnes par cimetière.
On peut même les repérer sur un plan.

Au 14 janvier 2023 seul le cimetière du Montparnasse est "complet". Celui du père Lachaise est quasiment finit. Aucun des deux ne prends en compte les monuments collectifs.

Pour certain·e·s communard·e·s, le lieu de sépulture (P119) est connu. Pour certain·e·s les coordonnées géographiques de la tombe sont dans wikidata.

> lieu de sépulture (P119)
lieu où le corps du sujet est conservé

## Nombre de personne par cimetière

Pour chaque communard·e·s cherche le lieu de sépulture et les coorodonnées de celles-ci. Compte le nombre de personnes par lieux.

```sparql
SELECT ?lieu_de_sépulture ?lieu_de_sépultureLabel ?coordonnees (COUNT(*) AS ?count) WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
 ?communard_ou_communarde wdt:P106 wd:Q1780490;
                           wdt:P119 ?lieu_de_sépulture.
 ?lieu_de_sépulture wdt:P31/wdt:P279* wd:Q39614;
                    wdt:P625 ?coordonnees.
}

group by ?lieu_de_sépulture ?lieu_de_sépultureLabel ?coordonnees
order by desc (?count)
```

### En sparql

In [2]:
%%HTML
<iframe style="width: 95%; height: 50vh; border: none;" 
src="https://query.wikidata.org/embed.html#SELECT%20%3Flieu_de_s%C3%A9pulture%20%3Flieu_de_s%C3%A9pultureLabel%20%3Fcoordonnees%20(COUNT(*)%20AS%20%3Fcount)%20WHERE%20%7B%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%0A%20%3Fcommunard_ou_communarde%20wdt%3AP106%20wd%3AQ1780490%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP119%20%3Flieu_de_s%C3%A9pulture.%0A%20%3Flieu_de_s%C3%A9pulture%20wdt%3AP31%2Fwdt%3AP279*%20wd%3AQ39614%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP625%20%3Fcoordonnees.%0A%7D%0A%0Agroup%20by%20%3Flieu_de_s%C3%A9pulture%20%3Flieu_de_s%C3%A9pultureLabel%20%3Fcoordonnees%0Aorder%20by%20desc%20(%3Fcount)" 
referrerpolicy="origin" sandbox="allow-scripts allow-same-origin allow-popups">
</iframe>

On peut afficher la carte avec la localisation des différents lieux

### En python

In [3]:
query = """SELECT ?lieu_de_sépulture ?lieu_de_sépultureLabel ?coordonnees (COUNT(*) AS ?count) WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
 ?communard_ou_communarde wdt:P106 wd:Q1780490;
                           wdt:P119 ?lieu_de_sépulture.
 ?lieu_de_sépulture wdt:P31/wdt:P279* wd:Q39614;
                    wdt:P625 ?coordonnees.
}

group by ?lieu_de_sépulture ?lieu_de_sépultureLabel ?coordonnees
order by desc (?count)
"""
extraction_lieu_sepulture = Extraction_wikidata(endpoint_url, query)
df_lieu_sepulture = extraction_lieu_sepulture.extraire_et_df()

In [4]:
df_lieu_sepulture = df_lieu_sepulture.loc[:,['lieu_de_sépulture.value', 'lieu_de_sépultureLabel.value', 'count.value', 'coordonnees.value']]
HTML(df_lieu_sepulture.to_html())

,lieu_de_sépulture.value,lieu_de_sépultureLabel.value,count.value,coordonnees.value
0,http://www.wikidata.org/entity/Q311,cimetière du Père-Lachaise,41,Point(2.394166666 48.861944444)
1,http://www.wikidata.org/entity/Q110481596,cimetière des condamnés,8,Point(167.443563 -22.636549)
2,http://www.wikidata.org/entity/Q272208,cimetière du Montparnasse,7,Point(2.32833 48.83778)
3,http://www.wikidata.org/entity/Q746647,cimetière de Montmartre,2,Point(2.330277777 48.887777777)
4,http://www.wikidata.org/entity/Q2972508,cimetière d'Auteuil,2,Point(2.25944 48.8411)
5,http://www.wikidata.org/entity/Q437930,cimetière ancien de Neuilly-sur-Seine,1,Point(2.264444444 48.881666666)
6,http://www.wikidata.org/entity/Q781858,cimetière de Charonne,1,Point(2.40347 48.8609)
7,http://www.wikidata.org/entity/Q779367,cimetière d'Ixelles,1,Point(4.392778 50.815556)
8,http://www.wikidata.org/entity/Q2972531,cimetière de Levallois-Perret,1,Point(2.289 48.9)
9,http://www.wikidata.org/entity/Q252312,cimetière du Nord de Solna,1,Point(18.019166666 59.356666666)


- [ ] Todo pour plus tard : afficher la carte

## Cimetière du Montparnasse
Cartes des tombes de communard·e·s ou de personnes ayant un lien avec la Commune dans le cimetière du Montparnasse (Q272208).

```sparql
#defaultView:Map
SELECT Distinct ?tombe ?tombeLabel ?image_Tombe ?coordonnées ?tombe_deLabel ?image_personne WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  
  #Qui, les communard-e-s et la liste des personnes lors de la cérémonie
  ?communard wdt:P106 wd:Q1780490;
             wdt:P119 wd:Q272208.
  Values ?personnalite {wd:Q313709 
                        wd:Q61897268 
                        wd:Q5749
                        wd:Q1959369
                        wd:Q3047711
                        wd:Q3022660
                        wd:Q551648
                        wd:Q704649
                        wd:Q2826417
                        wd:Q3302715
                        wd:Q337628}
  #Chercher les tombes du cimetière montparnasse                      
  ?tombe wdt:P361 wd:Q272208;
         wdt:P547 ?tombe_de.        
  #Prendre les tombes voulues
  filter (?tombe_de =?communard || ?tombe_de=?personnalite)
  
  OPTIONAL {?tombe wdt:P18 ?image_Tombe.}
  OPTIONAL {?tombe wdt:P625 ?coordonnées.}
  OPTIONAL {?tombe_de wdt:P18 ?image_personne.}
  
}
```

In [5]:
%%HTML
<iframe style="width: 95%; height: 75vh; border: none;" 
src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0ASELECT%20Distinct%20%3Ftombe%20%3FtombeLabel%20%3Fimage_Tombe%20%3Fcoordonn%C3%A9es%20%3Ftombe_deLabel%20%3Fimage_personne%20WHERE%20%7B%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%0A%20%20%0A%20%20%23Qui%2C%20les%20communard-e-s%20et%20la%20liste%20des%20personnes%20lors%20de%20la%20c%C3%A9r%C3%A9monie%0A%20%20%3Fcommunard%20wdt%3AP106%20wd%3AQ1780490%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP119%20wd%3AQ272208.%0A%20%20Values%20%3Fpersonnalite%20%7Bwd%3AQ313709%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ61897268%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ5749%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ1959369%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ3047711%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ3022660%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ551648%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ704649%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ2826417%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ3302715%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wd%3AQ337628%7D%0A%20%20%23Chercher%20les%20tombes%20du%20cimeti%C3%A8re%20montparnasse%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%3Ftombe%20wdt%3AP361%20wd%3AQ272208%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP547%20%3Ftombe_de.%20%20%20%20%20%20%20%20%0A%20%20%23Prendre%20les%20tombes%20voulues%0A%20%20filter%20(%3Ftombe_de%20%3D%3Fcommunard%20%7C%7C%20%3Ftombe_de%3D%3Fpersonnalite)%0A%20%20%0A%20%20OPTIONAL%20%7B%3Ftombe%20wdt%3AP18%20%3Fimage_Tombe.%7D%0A%20%20OPTIONAL%20%7B%3Ftombe%20wdt%3AP625%20%3Fcoordonn%C3%A9es.%7D%0A%20%20OPTIONAL%20%7B%3Ftombe_de%20wdt%3AP18%20%3Fimage_personne.%7D%0A%20%20%0A%7D" 
referrerpolicy="origin" 
sandbox="allow-scripts allow-same-origin allow-popups">
</iframe>

## Cimetière du Père la Chaise
Cartes des tombes de communard·e·s ou de personnes ayant un lien avec la Commune dans le cimetière du Père-Lahaise (Q311).

```sparql
#defaultView:Map
SELECT Distinct ?tombe ?tombeLabel ?image_Tombe ?coordonnées ?tombe_deLabel ?image_personne WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  
  #Qui, les communard-e-s
  ?communard wdt:P106 wd:Q1780490;
             wdt:P119 wd:Q311.

  #Chercher les tombes du cimetière montparnasse                      
  ?tombe wdt:P276 ?lieu;
         wdt:P547 ?tombe_de. 
  
  ?lieu wdt:P361 wd:Q311.
        
  #Prendre les tombes voulues
  filter (?tombe_de =?communard)
  
  OPTIONAL {?tombe wdt:P18 ?image_Tombe.}
  OPTIONAL {?tombe wdt:P625 ?coordonnées.}
  OPTIONAL {?tombe_de wdt:P18 ?image_personne.}
  
}
```

In [6]:
%%HTML
<iframe style="width: 95%; height: 50vh; border: none;" 
src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0ASELECT%20Distinct%20%3Ftombe%20%3FtombeLabel%20%3Fimage_Tombe%20%3Fcoordonn%C3%A9es%20%3Ftombe_deLabel%20%3Fimage_personne%20WHERE%20%7B%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%0A%20%20%0A%20%20%23Qui%2C%20les%20communard-e-s%0A%20%20%3Fcommunard%20wdt%3AP106%20wd%3AQ1780490%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP119%20wd%3AQ311.%0A%0A%20%20%23Chercher%20les%20tombes%20du%20cimeti%C3%A8re%20montparnasse%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%3Ftombe%20wdt%3AP276%20%3Flieu%3B%0A%20%20%20%20%20%20%20%20%20wdt%3AP547%20%3Ftombe_de.%20%0A%20%20%0A%20%20%3Flieu%20wdt%3AP361%20wd%3AQ311.%0A%20%20%20%20%20%20%20%20%0A%20%20%23Prendre%20les%20tombes%20voulues%0A%20%20filter%20(%3Ftombe_de%20%3D%3Fcommunard)%0A%20%20%0A%20%20OPTIONAL%20%7B%3Ftombe%20wdt%3AP18%20%3Fimage_Tombe.%7D%0A%20%20OPTIONAL%20%7B%3Ftombe%20wdt%3AP625%20%3Fcoordonn%C3%A9es.%7D%0A%20%20OPTIONAL%20%7B%3Ftombe_de%20wdt%3AP18%20%3Fimage_personne.%7D%0A%20%20%0A%7D%0A"referrerpolicy="origin" 
sandbox="allow-scripts allow-same-origin allow-popups">
</iframe>


## Licence

Ce travail est sous licence libre plus précisement Creative Commons Attribution - Partage dans les Mêmes Conditions 4.0 International.

Il est fait à partir de données libres (diverse licence) et logiciels ou formats libres.
Citons python, notebook, makedon, mediawiki...
J'autorise la diffusion commerciale pour payer les frais de papier si quelqu'an veut le diffuser en version papier.
Par principe, la licence vous autorise à utiliser ce travail sans m'en informer, juste pour ma curiosité, merci de me le signaler.

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licence Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />Cette œuvre est mise à disposition selon les termes de la <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Licence Creative Commons Attribution -  Partage dans les Mêmes Conditions 4.0 International</a>.

La version originale du notebook et des fichiers liées sont versionnés et héberger sur github. 
https://github.com/silanoc/commune_dans_wikidata